<img src='h1.png'></img>

<img src='h2.png'></img>

In [1]:
import numpy as np
from scipy.special import binom as C

In [2]:
M = 100 #число хромосом
mu = 2 #частота мутаций
K = 10 #количество ответвлений, считая от изначального предка к конечным потомкам = количество конечных потомков

In [3]:
T = np.array([0, .04, .01, .07], dtype=float) #периоды времени
N = np.array([2, 5, 10, 6], dtype=int) #размеры популяции

In [4]:
def v(t):
    '''
    размер популяции в данный момент времени t
    '''
    cond = list()
    for i in range(len(T) - 1):
        cond.append(T[i] <= t < T[i + 1])
    cond.append(t >= T[-1])
    return np.piecewise(t, cond, N)

In [5]:
def w(K, t):
    '''
    интенсивность процесса Пуассона
    K - количество конечных потомков
    t - время
    '''
    return K * mu + 1 / v(t) * C(K, 2)

In [6]:
def add_chromosome(lineage, pos, mutated):
    '''
    добавление хромосомы к особи и её потомкам
    lineage - ветка предок-потомки
    pos - позиция в геноме - хромосома, которую мы хотим добавить
    mutated - признак мутации: 1 - есть, 0 - нет
    '''
    #заходим в нулевой элемент списка - это кортеж. В нём первый эемелент - это словарь.
    lineage[0][1][pos] = mutated
    #если список lineage содержит потомка или двоих, повторяем действие
    for i in range(1, len(lineage)):
        add_chromosome(lineage[i], pos, mutated)

In [7]:
def simulation(K, mu, T, N):
    '''
    коалесцентная модель
    K - количество конечных потомков
    mu - частота мутаций
    T - периоды времени
    N - размеры популяции
    
    возвращает дерево в виде вложенных списков и времена Пуассоновского процесса для мутации и коалесценции
    '''
    decendents = list([(i, {})] for i in range(K)) #основа будущего дерева
    np.random.seed(1)
    t = 0
    t_mut = list()
    t_coal = list()
    while K > 1:
        W = w(K, t)
        weight = K * mu
        
        '''
        генерация события мутации, используя распределение Бернулли
        вес weight = K * mu
        вероятность weight / w(K, t)
        
        используется биномиальное распределение с числом испытаний = 1,
        поэтому количество успехов может быть равно либо 1, либо 0
        
        из определения распределения Бернулли неудача события мутации даёт успех события коалесценции 
        '''
        if np.random.binomial(1, weight / W):
            t_mut.append(t)
            #случайно выбираем одного потомка, который оказался подвержен мутации
            l = np.random.randint(0, K)
            #случайно выбираем хромосому, которая оказалась подвержена мутации
            p = np.random.random()
            #всем потомкам особи l передаём эту мутацию - работем внутри ветки decendents[l]
            add_chromosome(decendents[l], p , 1)
            #остальные ветки получают ноль
            for i in range(l):
                add_chromosome(decendents[i], p, 0)
            for i in range(l + 1, K):
                add_chromosome(decendents[i], p, 0)
            print('K =', K)
            print('t = ', t)
            print('мутация хромосомы', p, 'у особи', l)
            print('--------------------------------')
        else:
            t_coal.append(t)
            l1, l2 = np.random.choice(K, 2, replace=False)
            lmin = min(l1, l2) 
            lmax = max(l1, l2)
            
            print('K =', K)
            print('t = ', t)
            print('коалесценция между особями', l1, 'и', l2)
            print('--------------------------------')
            K -= 1
            ancestors = list([(i, {})] for i in range(K))
            for i in range(lmin):
                ancestors[i].append(decendents[i])
            #чтобы провести даньнейшую связь предков и потомков, проведём замену
            decendents[lmin + 1], decendents[lmax] = decendents[lmax], decendents[lmin + 1]
            ancestors[lmin].append(decendents[lmin])
            ancestors[lmin].append(decendents[lmin + 1])
            for i in range(lmin + 1, K):
                ancestors[i].append(decendents[i + 1])
            decendents = ancestors
        
        '''
        генерация времени до следующего события Пуассоновского процесса
        время имеет экспоненциальное распределение
        параметр равен интенсивности Пуассоновского процесса, т.е. w(K,t)
        '''
        t += np.random.exponential(1 / W)
        
    return decendents, t_mut, t_coal

In [8]:
tree = simulation(K, mu, T, N)[0]

K = 10
t =  0
коалесценция между особями 2 и 3
--------------------------------
K = 9
t =  0.011893000981429376
мутация хромосомы 0.6852195003967595 у особи 5
--------------------------------
K = 9
t =  0.022482093922486335
коалесценция между особями 7 и 3
--------------------------------
K = 8
t =  0.18221483511275033
мутация хромосомы 0.518152548941889 у особи 6
--------------------------------
K = 8
t =  0.2791163121993391
коалесценция между особями 6 и 4
--------------------------------
K = 7
t =  0.32262130018888563
мутация хромосомы 0.5331652849730171 у особи 1
--------------------------------
K = 7
t =  0.3898931056117375
мутация хромосомы 0.7833144726948862 у особи 4
--------------------------------
K = 7
t =  0.4202899712315111
мутация хромосомы 0.9888610889064947 у особи 2
--------------------------------
K = 7
t =  0.49908904359012823
мутация хромосомы 0.07336417174259957 у особи 2
--------------------------------
K = 7
t =  0.5352857613611859
мутация хромосомы 0.28777533858

In [9]:
tree

[[(0, {}),
  [(0, {0.5688514370864813: 0, 0.6199557183813798: 1, 0.6511205589027862: 0}),
   [(0,
     {0.01864728937294302: 1,
      0.028306488020794607: 1,
      0.0915564416778033: 0,
      0.15213715839256237: 0,
      0.2943594751304114: 1,
      0.518861875863902: 0,
      0.5528219786857659: 0,
      0.5562402339904189: 0,
      0.5688514370864813: 0,
      0.6129480927278149: 0,
      0.6199557183813798: 1,
      0.6511205589027862: 0,
      0.7129889803826767: 0,
      0.7554630526024664: 0,
      0.9371903244000067: 0}),
    [(0,
      {0.01864728937294302: 1,
       0.028306488020794607: 1,
       0.0915564416778033: 0,
       0.15213715839256237: 0,
       0.2943594751304114: 1,
       0.32664490177209615: 0,
       0.470640807662421: 0,
       0.518861875863902: 0,
       0.5528219786857659: 0,
       0.5562402339904189: 0,
       0.5688514370864813: 0,
       0.6129480927278149: 0,
       0.6199557183813798: 1,
       0.6511205589027862: 0,
       0.7129889803826767: 0,


### Коалесценция
На каждом поколении своя нумерация. Принадлежность потомка непосредственному предку обозначается их нахождением в одном списке, причём потомок имеет на один уровень вложенности больше.

Например:

[0, [1] ] - предок 0 со своим потомком 1.

[0, [1, [1], [2] ] ] - предок 0 со своим потомком 1, у которого потомки 1 и 2.

### Добавляем мутацию
Теперь каждая особь это не число, а кортеж из двух элементов. Первый элемент - особь. Второй - словарь. Ключи словаря - хромосомы. Значение каждого ключа - 0 или 1 - наличие мутации у данной хромосомы.

Например:

[(0, {}), [(1, {0.5: 1})] ] - предок 0 со своим потомком 1. У потомка мутация хромосомы 0.5

[(0, {}), 

    [(1, {0.5: 1}), 
        [(1, {0.5: 1, 0.67: 0})], 
        [(2, {0.5: 1, 0.67: 1})] 
    ] 
]

предок 0 со своим потомком 1 (мутация 0.5), у которого потомки 1 (мутация 0.5; 0.67 без мутации) и 2 (мутация 0.5 и 0.67).